In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('data/reviews.csv')

In [ ]:
df.head(10)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df_train, df_test = train_test_split(df, test_size=0.5)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support as score

In [ ]:
text_clf = Pipeline([
    ('vect', CountVectorizer(ngram_range=(1,1))),
    ('tfidf', TfidfTransformer()),
    ('clf', RandomForestClassifier())
    ])

In [ ]:
text_type = 'alpha_text'

In [ ]:
text_clf.fit(df_train[text_type], df_train['review'])

In [ ]:
text_clf['vect'].get_feature_names()[100:110]

In [ ]:
len(text_clf['vect'].get_feature_names())

In [ ]:
text_clf['tfidf'].idf_

In [ ]:
text_clf['clf']

In [ ]:
text_clf['clf'].feature_importances_

In [ ]:
feature_df = pd.DataFrame({"feature": text_clf['vect'].get_feature_names(), 
                           'importance': text_clf['clf'].feature_importances_})

In [ ]:
features_sorted = feature_df.sort_values(by=['importance'], ascending=False)

In [ ]:
features_sorted.head(10)

In [ ]:
len(features_sorted)

In [ ]:
y_pred = text_clf.predict(df_test[text_type])

In [ ]:
#y_pred

In [ ]:
precision, recall, fscore, train_support = score(df_test['review'], y_pred, pos_label=1, average='binary')
print('Precision: {} / Recall: {} / Accuracy: {}'.format(
    round(precision, 3), round(recall, 3), round((y_pred==df_test['review']).sum()/len(y_pred), 3)))

In [ ]:
# body_text - Precision: 0.735 / Recall: 0.742 / Accuracy: 0.745
# alpha_text - Precision: 0.788 / Recall: 0.65 / Accuracy: 0.74
# nostop_text - Precision: 0.799 / Recall: 0.704 / Accuracy: 0.76
# lemmatized_text - Precision: 0.81 / Recall: 0.67 / Accuracy: 0.753

In [ ]:
df_test['pred'] = y_pred

In [ ]:
y_proba = text_clf.predict_proba(df_test[text_type])

In [ ]:
y_proba

In [ ]:
pos_proba = [item[1] for item in y_proba]

In [ ]:
pos_proba

In [ ]:
df_test['proba'] = pos_proba

In [ ]:
df_test

In [ ]:
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

In [ ]:
df_test.set_index('row_id')

In [ ]:
pysqldf("SELECT " + text_type + ", review, pred, proba FROM df_test WHERE review = 0 \
        AND review != pred ORDER BY proba DESC")

In [ ]:
pysqldf("SELECT * FROM df_test ORDER BY proba asc")

In [ ]:
df_test.set_index("row_id")

In [ ]:
df_test.loc[741]['body_text']